## Подготовка данных

In [1]:
import pip
pip.main(['install', 'pymorphy2'])

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


0

In [2]:
import re

import pandas as pd
import pymorphy2


text_df = pd.read_csv("content_description.csv", sep='\t')
text_df.head()

# разбиваем тест на слова
corpus = []
# регулярка для поиска слов
regular_expr = r'\w+'
reg_expr_compiled = re.compile(regular_expr)
# формируем датасет из отдельных слов
for raw_text in text_df.description.values:
    # приводим к нижнему регистру
    raw_text_lower = raw_text.lower()
    # разбиваем текст на слова
    text_by_words = reg_expr_compiled.findall(raw_text_lower) 
    corpus.append(text_by_words)

# нормализация текста
normalized_corpus = []
morph = pymorphy2.MorphAnalyzer()
# нормализуем каждое слово в тексте
for token_list in corpus:
    normalized_token_list = []
    for word in token_list:
        parsed_token = morph.parse(word)
        normal_form = parsed_token[0].normal_form
        normalized_token_list.append(normal_form)
    normalized_corpus.append(normalized_token_list)

# превращаем в DataFrame
doc_count = len(normalized_corpus)
doc_ids = []
tokens = []
# формируем два списка-колонки датафрейма
for doc_id in range(doc_count):
    for token in normalized_corpus[doc_id]:
        doc_ids.append(doc_id)
        tokens.append(token)

tokens_df = pd.DataFrame({
    'doc_id': doc_ids,
    'word': tokens
})
# дамми-столбец
tokens_df = tokens_df.assign(dummy = 1)
# аггрегируем статистики
word_count_df = tokens_df.groupby(['doc_id','word'])['dummy'].count().reset_index()

word_count_df[word_count_df.doc_id==0].sort_values(by='dummy', ascending=False).head(10)

Loading dictionaries from D:\anaconda3\lib\site-packages\pymorphy2_dicts\data
format: 2.4, revision: 393442, updated: 2015-01-17T16:03:56.586168


,doc_id,word,dummy
6,0,в,10
28,0,и,7
36,0,который,5
110,0,шерлок,4
83,0,сериал,4
76,0,с,4
8,0,весь,3
74,0,риколетти,3
104,0,холмс,2
22,0,же,2


10.3.1 **Задание простого уровня** Для каждого слова подсчитайте процент документов, в которых содержится это слово. Сформируйте dataframe doc_frequency. Процен документов вычисляется по формуле
$$
d = \frac{m}{n} \times 100
$$
где $m$ - количество документов, в которых встретилось это слово, а $n$ - общее количество документов. Значение в процентах округлите до целых.

Посмотрите какие слова оказались в топе - это предлоги и частицы. Такие слова присутствуют во всех документах, а значит, их можно удалить из текста, чтобы оставшиеся слова были более "осмысленными". Это упростит анализ текста.

In [83]:
doc_frequency = word_count_df.groupby('word')['doc_id'].count().reset_index()
doc_count = len(word_count_df['doc_id'].unique())
doc_frequency['frequency'] = doc_frequency['doc_id'] / doc_count * 100
doc_frequency.sort_values(by = 'frequency', ascending=False).head()

,word,doc_id,frequency
172,и,8,100.0
41,в,8,100.0
270,на,7,87.5
468,с,7,87.5
324,он,6,75.0


In [84]:
pos = {'INTJ', 'PRCL', 'CONJ', 'PREP', 'NPRO', 'UNKN'}
words_list = doc_frequency['word'].tolist()
words_list_trimmed = [word for word in words_list if morph.parse(word)[0].tag.POS in pos] + ['свой', 'самый', 'тот', 'весь', 'который', 'этот', 'сам', 'наш', 'можно', 'мочь', 'вскоре', 'где', 'теперь']
doc_frequency_trimmed = doc_frequency.loc[~doc_frequency['word'].isin(words_list_trimmed)]
doc_frequency_trimmed.sort_values(by = 'frequency', ascending=False).head()

,word,doc_id,frequency
327,онлайн,6,75.0
498,смотреть,5,62.5
91,год,5,62.5
192,история,3,37.5
453,режиссёр,3,37.5


10.3.2 **Задание среднего уровня**

Воспользуйтесь регулярными выражениями, чтобы извлечь из текста все пары **имя+фамилия**.

* неформальное описание регулярки: пара слов идущая друг за другом, каждое из которых начинается с заглавной буквы
* анализировать нужно только `doc_id=3`
* текст берём из исходного датафрейма `text_df`
* заглавная буква в русскоязычном тексте соответствует символьному классу `r'[A-Я]*'`

In [85]:
# -- ВАШ КОД ЗДЕСЬ --
raw_text = text_df.description.values[3]
# регулярка - её нужно поправить
reg_expr = r'[^\.]\s([A-Я]{1}\w+\s[A-Я]{1}\w+)'
# компилируем регулярное выражение
reg_expr_compiled = re.compile(reg_expr)
# применяем выражение к тексту
for g in reg_expr_compiled.findall(raw_text):
    print(g)

Стивена Фрая
Джоном Дженксом
Теда Уоллеса


10.3.3 **Задание сложного уровня**. Словарь `genre_dict` содержит слова, которые являются характерными для того или иного жанра. Пользуясь словарём, а так же таблицей `word_count_df`, сформируйте таблицу двумя колонками `doc_id | genre` с жанрами фильмов.

Для этого нужно
* превратить словарь `genre_dict` в DataFrame формата `word | genre`
* соединить полученный датафрейм с помощью функции `merge` c ранее полученным датафреймом `word_count_df`, котороый содержит распределение слов по документам. Воспользуйтесь методом соединения `inner`
* для каждого документа выбрать жанр документа - это совокупность жанров слов отдельных слов. У одного контента может быть несколько жанров

In [86]:
word_count_df.head()

,doc_id,word,dummy
0,0,1895,1
1,0,а,1
2,0,абсолютно,1
3,0,англия,1
4,0,безобразный,1


In [87]:
import itertools
import pandas as pd

genre_dict = {
    'комедия': ['сатирический', 'авантюрный', 'забавный'],
    'мелодрама': ['выбор', 'позор'],
    'сказка': ['приключения', 'милый', 'семейный'],
    'детектив': ['тайна', 'разгадать', 'загадочный'], 
    'триллер': ['ужас', 'зловещий', 'нерв']
}

nested_genres =[[(i, j) for j in genre_dict[i]] for i in genre_dict]
# print(nested_genres)
flatten_genres = list(itertools.chain(*nested_genres))
# print(flatten_genres)
genres_df = pd.DataFrame(flatten_genres, columns = ['genre', 'word'])


In [88]:
# result = genres_df.merge(word_count_df)
result = pd.merge(genres_df, word_count_df, how='inner', on='word')
result

,genre,word,doc_id,dummy
0,комедия,сатирический,3,1
1,комедия,авантюрный,4,1
2,комедия,забавный,4,1
3,мелодрама,выбор,2,1
4,мелодрама,позор,2,1
5,сказка,милый,1,1
6,детектив,тайна,0,1
7,детектив,разгадать,0,1
8,детектив,загадочный,0,1
9,триллер,ужас,0,1


In [89]:
doc_to_genres = result.groupby('doc_id')['genre'].apply(lambda x: ', '.join(set(x))).reset_index()
doc_to_genres

,doc_id,genre
0,0,"триллер, детектив"
1,1,сказка
2,2,мелодрама
3,3,комедия
4,4,комедия
5,6,триллер
